# bge-large-zh

## curl 简单调用 -- 判断是否可用

In [1]:
%%time
%%capture
%%bash

curl -s http://192.168.0.72:11435/api/embeddings \
    -H "Content-Type: application/json" \
    -d '{
        "model": "dztech/bge-large-zh:v1.5",
        "prompt": "Llamas are members of the camelid family"
    }'

CPU times: user 5.26 ms, sys: 137 µs, total: 5.39 ms
Wall time: 32 ms


## LlamaIndex api - 生成嵌入向量

In [2]:
%%time

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], 
    # show_progress=True
)
len(pass_embedding[1])

CPU times: user 1.39 s, sys: 248 ms, total: 1.64 s
Wall time: 1.28 s


1024

## LlamaIndex api + Faiss存储索引

### 加载库

In [3]:
%%time
%%capture

!pip install llama-index
!pip install llama-index-llms-openai-like
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-faiss

CPU times: user 25.1 ms, sys: 21 ms, total: 46 ms
Wall time: 7.38 s


In [4]:
%%time

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 2.47 ms


### 设置系统日志

In [5]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 249 µs, sys: 0 ns, total: 249 µs
Wall time: 252 µs


### 嵌入模型 - bge-large-zh

In [6]:
%%time

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0},
)

Settings.embed_model = ollama_embedding

CPU times: user 635 ms, sys: 19.5 ms, total: 654 ms
Wall time: 654 ms


### 加载文档

In [7]:
%%time

documents = SimpleDirectoryReader("./books/").load_data()
documents

CPU times: user 9.67 ms, sys: 577 µs, total: 10.2 ms
Wall time: 9.84 ms


[Document(id_='990e7943-85a2-4c2d-90f8-ffeb95a34cab', embedding=None, metadata={'file_path': '/root/notebook/my-jupyter-notebook/llm/bge/books/iphone.txt', 'file_name': 'iphone.txt', 'file_type': 'text/plain', 'file_size': 563, 'creation_date': '2024-05-07', 'last_modified_date': '2024-05-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='iPhone 15和iPhone 15 Plus是由Apple Inc.设计、开发和销售的智能手机。它们是第十七代、也是当前一代iPhone，接替iPhone 14 和 iPhone 14 Plus。这些设备于 2023 年 9 月 12 日在加利福尼亚州库比蒂诺Apple Park举行的Apple 活动期间与价格较高的iPhone 15 Pro 和 15 Pro Max一起发布。预订于 2023 年 9 月 15 日开始，这些设备于 2023 年 9 月 22 日上市。与 iPhone 15 Pro 和 Pro Max 一样，15 和 15 Plus 是第一批用USB-C取代专有Lightning连接器的iPhone遵守欧盟的指令。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_te

### 创建Faiss索引

ModuleNotFoundError: No module named 'faiss.swigfaiss_avx2'

解决办法是：

```bash
# find / -name swigfaiss.py
/usr/local/lib/python3.10/dist-packages/faiss/swigfaiss.py
# cd /usr/local/lib/python3.10/dist-packages/faiss/
# ln -s swigfaiss.py swigfaiss_avx2.py
```

https://github.com/kyamagu/faiss-wheels/issues/39

In [9]:
%%time

import faiss

# dimensions of bge-large-zh
d = 1024
faiss_index = faiss.IndexFlatL2(d)

INFO:faiss.loader:Loading faiss with AVX2 support.
Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.
CPU times: user 53.1 ms, sys: 8.71 ms, total: 61.8 ms
Wall time: 61.7 ms


In [10]:
%%time

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 225 ms, sys: 7.98 ms, total: 233 ms
Wall time: 309 ms


In [11]:
%%time

# save index to disk
index.storage_context.persist()

CPU times: user 2.07 ms, sys: 269 µs, total: 2.34 ms
Wall time: 1.91 ms


In [12]:
%%time

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

INFO:root:Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
CPU times: user 5.3 ms, sys: 164 µs, total: 5.46 ms
Wall time: 4.49 ms


### 查询索引

In [13]:
%%time

from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True
                )
Settings.llm =llm

CPU times: user 1.08 s, sys: 144 ms, total: 1.23 s
Wall time: 1.23 s


In [18]:
%%time

# set Logging to DEBUG for more detailed outputs
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    similarity_top_k=3,
    similarity_cutoff=0.5
)
response = query_engine.query("介绍下iphone15")

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
CPU times: user 12.1 ms, sys: 3.51 ms, total: 15.6 ms
Wall time: 6.66 s


In [20]:
%%time

from IPython.display import Markdown, display

display(Markdown(f"{response}"))

iPhone 15和iPhone 15 Plus是由Apple Inc.设计、开发和销售的智能手机。它们是第十七代、也是当前一代iPhone，接替iPhone 14 和 iPhone 14 Plus。这些设备于 2023 年 9 月 12 日在加利福尼亚州库比蒂诺Apple Park举行的Apple 活动期间与价格较高的iPhone 15 Pro 和 15 Pro Max一起发布。预订于 2023 年 9 月 15 日开始，这些设备于 2023 年 9 月 22 日上市。与 iPhone 15 Pro 和 Pro Max 一样，15 和 15 Plus 是第一批用USB-C取代专有Lightning连接器的iPhone，遵守欧盟的指令。

CPU times: user 1.89 ms, sys: 177 µs, total: 2.07 ms
Wall time: 1.73 ms
